<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Исследовательский-анализ-данных" data-toc-modified-id="Исследовательский-анализ-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследовательский анализ данных</a></span></li><li><span><a href="#Корреляционный-анализ-признаков" data-toc-modified-id="Корреляционный-анализ-признаков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Корреляционный анализ признаков</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-LinearRegression" data-toc-modified-id="Модель-LinearRegression-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Модель LinearRegression</a></span></li><li><span><a href="#Модель-DecisionTree" data-toc-modified-id="Модель-DecisionTree-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Модель DecisionTree</a></span></li><li><span><a href="#Модель-LightGBM" data-toc-modified-id="Модель-LightGBM-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Модель LightGBM</a></span></li><li><span><a href="#Модель-CatBoost" data-toc-modified-id="Модель-CatBoost-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Модель CatBoost</a></span></li></ul></li><li><span><a href="#Отчёт-по-исследованию" data-toc-modified-id="Отчёт-по-исследованию-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Отчёт по исследованию</a></span></li></ul></div>

# Проект: разработка модели для предсказания рыночной стоимости автомобиля

Описание проекта: сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля.

Задача - построить модель, которая умеет её определять. Для этого у нас есть данные о технических характеристиках, комплектации и ценах других автомобилей.
Критерии, которые важны заказчику:
- качество предсказания;
- время обучения модели;
- время предсказания модели.

Примечание: для оценки качества моделей применим метрику RMSE, её значение должно быть меньше 2500.

## Загрузка данных

Загрузим необходимые библиотеки:

In [ ]:
! pip install matplotlib==3.7.1 -q
! pip install -U scikit-learn -q

In [ ]:
pip install catboost

In [ ]:
pip install lightgbm

In [ ]:
pip install phik

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import datetime
import warnings
warnings.filterwarnings("ignore")

import phik
from phik import phik_matrix
from phik.report import plot_correlation_matrix

from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

import sklearn
from sklearn.model_selection import(
    KFold,
    cross_val_score,
    train_test_split,
    RandomizedSearchCV,
    GridSearchCV
)
from sklearn.preprocessing import(
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler
)
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree, \
    DecisionTreeRegressor

In [ ]:
RANDOM_STATE = 1897
TEST_SIZE = 0.25

In [ ]:
path_1 = 'autos.csv'
path_2 = '/datasets/autos.csv'

if os.path.exists(path_1):
    df = pd.read_csv(path_1)
elif os.path.exists(path_2):
    df = pd.read_csv(path_2)
else:
    print('Something is wrong!')

Проверим, что данные загрузились корректно:

In [ ]:
# настройка для вывода всех столбцов в датасете:
pd.set_option('display.max_columns', None)

In [ ]:
# функция для получения информации о датасете
def get_info(dataset):
    print('Первые пять строк датасета:')
    display(dataset.head())
    print('Общая информация:')
    dataset.info()

In [ ]:
get_info(df)

Столбцы, содержащие даты, нуждаются в корректировке типов данных:

In [ ]:
df['DateCrawled'] = pd.to_datetime(df['DateCrawled'])
df['DateCreated'] = pd.to_datetime(df['DateCreated'])
df['LastSeen'] = pd.to_datetime(df['LastSeen'])

Приведём к единому виду названия столбцов (snake_case, строчные буквы). Заодно переименуем столбец `Kilometer` в более подходящее по смыслу название `car_mileage`:

In [ ]:
df = df.rename(columns={'DateCrawled': 'date_crawled', 'Price': 'price', \
      'VehicleType': 'vehicle_type', 'RegistrationYear': 'registration_year', \
      'Gearbox': 'gearbox', 'Power': 'power', 'Model': 'model', \
      'Kilometer': 'car_mileage', 'RegistrationMonth': 'registration_month', \
      'FuelType': 'fuel_type', 'Brand': 'brand', 'Repaired': 'repaired', \
      'DateCreated': 'date_created', 'NumberOfPictures': 'number_of_pictures', \
      'PostalCode': 'postal_code', 'LastSeen': 'last_seen'})

In [ ]:
get_info(df)

Описание полей данных:

- `date_crawled` — дата скачивания анкеты из базы
- `price` - стоимость автомобиля в евро, <b>целевой признак</b>
- `vehicle_type` — тип автомобильного кузова
- `registration_year` — год регистрации автомобиля
- `gearbox` — тип коробки передач
- `power` — мощность автомобиля в л.с.
- `model` — модель автомобиля
- `car_mileage` — пробег автомобиля в км
- `registration_month` — месяц регистрации автомобиля
- `fuel_type` — тип топлива
- `brand` — марка автомобиля
- `repaired` — был автомобиль в ремонте или нет
- `date_created` — дата создания анкеты
- `number_of_pictures` — количество фотографий автомобиля
- `postal_code` — почтовый индекс владельца анкеты (пользователя)
- `last_seen` — дата последней активности пользователя

Выводы по разделу:

- изучили данные из предоставленного датасета, привели названия столбцов к единому виду (snake_case, строчные буквы). Переименовали название столбца `Kilometer` в более подходящее по смыслу `car_mileage`;
- у столбцов `date_crawled`, `date_created` и `last_seen` поменяли тип данных на `datetime`, соответственно их значениям.

Переходим к этапу предобработки данных.

## Предобработка данных

Проверим датасет на наличие пропусков и явных дубликатов, воспользовавшись созданной функцией:

In [ ]:
def data_check(dataset):
    print("\nПроверка на наличие пропусков в датасете\n")
    null = dataset.isnull().sum()
    if null.sum() > 0:
            print("Пропущенные значения в датасете:\n")
            print(null)
    else:
        print("Отсутствуют пропущенные значения в датасете\n")

    print("\nПроверка на наличие явных дубликатов в датасете\n")
    duplicates = dataset.duplicated().sum()
    if duplicates > 0:
        print(f"Количество явных дубликатов в датасете: {duplicates}")
    else:
        print("Отсутствуют явные дубликаты в датасете")

In [ ]:
data_check(df)

Выведем строки с явными дубликатами:

In [ ]:
duplicates = df[df.duplicated(keep=False)]
print(duplicates)

Как видим, это одинаковые автомобили (4 шт.), встречающиеся дважды. Вероятно, дублирование произошло случайно, поэтому удалим лишние строки:

In [ ]:
df.drop_duplicates(inplace=True)
# Проверим, что явные дубликаты удалены:
data_check(df)

Всего в датасете 354 369 записей, решение о заполнении пропусков примем позже, когда детальнее изучим данные, а сначала посмотрим на уникальные значения в категориальных столбцах:

In [ ]:
df['vehicle_type'].unique()

In [ ]:
df['gearbox'].unique()

In [ ]:
unique_models = df['model'].unique()
unique_models_sorted = pd.Series(unique_models).sort_values().tolist()
unique_models_sorted

Список содержит одинаковые модели, записанные по-разному: `range_rover` и `rangerover`. Приведём к единому виду:

In [ ]:
df['model'] = df['model'].replace('rangerover', 'range_rover')

In [ ]:
df['fuel_type'].unique()

`petrol` и `gasoline` - слова, означающие одно и то же - "бензин". Разница лишь в том, что термин "petrol" используют в Великобритании, Индии и ещё ряде стран, в то время как "gasoline" - в США. Для упрощения оставим лишь один термин в датасете, пусть им будет "petrol":

In [ ]:
df['fuel_type'] = df['fuel_type'].replace({'gasoline': 'petrol'})

In [ ]:
df['fuel_type'].unique()

In [ ]:
df['brand'].unique()

In [ ]:
df['repaired'].unique()

In [ ]:
df['number_of_pictures'].unique()

Любопытно, что датасет не содержит фотографий автомобилей, все строки - с нулевым их количеством

Выводы по разделу:

- проверили датасет на наличие пропущенных значений, заполним их позже;
- обнаружили 4 явных дубликата, удалили их;
- обнаружили одинаковые модели, записанные по-разному: `range_rover` и `rangerover`. Привели к единому виду:
- переименовали значение `gasoline` в `petrol` в столбце `fuel_type`, так как по сути это одно и то же. Разница - в стране употребления того или иного термина;
- неявные дубликаты обнаружены не были.

## Исследовательский анализ данных

Посмотрим на статистические данные количественных признаков датасета:

In [ ]:
df.describe().round(2)

Бросаются в глаза следующие аномальные значения:

- "нулевая" стоимость автомобиля в столбце `price`;
- 1000-й и 9999-й годы регистрации автомобиля в столбце `registration_year`;
- "нулевая" мощность автомобиля в столбце `power`. Согласно книге рекордов, максимальная мощность автомобиля 2300 л.с. (суперкар Koenigsegg Gemera), так что 20000 л.с. в датасете считаем аномалией;
- "нулевой" месяц регистрации автомобиля в столбце `registration_month`;
- "нулевое" количество фотографий, что уже подметили ранее.

Посмотрим на доли выбросов в интересующих нас количественных столбцах:

In [ ]:
def outliers(dataset, column):
    q1 = dataset[column].quantile(0.25) # первый квартиль
    q3 = dataset[column].quantile(0.75)  # третий квартиль
    iqr = q3 - q1  # межквартильный размах
    quant1 = q1 - 1.5*iqr  # нижняя граница выбросов
    quant3 = q3 + 1.5*iqr  # верхняя граница выбросов
    total = dataset.shape[0]  # общее количество строк в датафрейме
    emission = dataset[(dataset[column] < quant1) | \
     (dataset[column] > quant3)].shape[0]
    # количество выбросов
    emission_quant = (emission / total) * 100  # доля выбросов в столбце
    return np.round((emission_quant), decimals=2)
    # возвращаем долю выбросов в столбце

In [ ]:
dataset = df
column_list = ['price', 'registration_year', 'power', 'car_mileage',
        'registration_month']
for value in column_list:
    print(f'Доля выбросов в {value}, в процентах: {outliers(dataset, value)}')

Доли выбросов незначительные везде, кроме столбца `car_mileage`. Отобразим данные на гистограммах и диаграммах:

In [ ]:
# Гистограмма распределения количественных признаков
def histogram(dataset, column):
    dataset[column].plot(kind='hist', bins=20, grid=True, \
                figsize=(5, 5), title=(f'Распределение признака "{column}"'))
    plt.xlabel(column)
    plt.ylabel('Количество')
    plt.show()
    return None

# Диаграмма размаха
def whiskers(dataset, column):
    dataset.boxplot(column, figsize=(5, 5))
    plt.title(f'Диаграмма размаха признака "{column}"')
    plt.ylabel('Количество')
    plt.show()
    print(dataset[column].describe().round(2))
    return None

# Диаграмма распределения категориальных признаков
def categorial(dataset, column):
    dataset[column].value_counts().plot(
    kind = 'pie',
    ylabel = '',
    autopct = '%1.1f%%',
    figsize = (5,5),
    title = f'Процентное распределение признака "{column}"')
    plt.show()
    return None

In [ ]:
histogram(df, 'price')
whiskers(df, 'price')

Маловероятно, что автомобиль может стоить дешевле даже 1000 евро (25% выборки), тем не менее, посмотрим, какую долю занимают автомобили за такую стоимость:

In [ ]:
print(round(df[df.price < 1000].shape[0] / df.shape[0] * 100, 2))

Большой процент, но будем считать данные по такой стоимости ошибочными, удалим их:

In [ ]:
df = df[(df.price >= 1000)]

In [ ]:
histogram(df, 'registration_year')
whiskers(df, 'registration_year')

Вряд ли мы найдём автомобили, выпущенные в 1000-м году, как и выпущенные позже даты скачивания анкеты из базы (4 июля 2016 г.). Удалим данные, не лежащие в интервале 1900 - 2017 гг.:

In [ ]:
df = df[(df.registration_year >= 1900) & (df.registration_year <= 2017)]

In [ ]:
histogram(df, 'power')
whiskers(df, 'power')

In [ ]:
print(round(df[df.power < 50].shape[0] / df.shape[0] * 100, 2))

In [ ]:
print(round(df[df.power > 2300].shape[0] / df.shape[0] * 100, 2))

В списке автомобилей из датасета отсутствуют даже 10 самых маломощных автомобилей (16 л.с. и менее), 25% выборки лежит в пределах 75 л.с. Самого мощного автомобиля в мире (2300 л.с.) здесь также нет. Ограничим датасет пределами 50 - 2300 л.с., чтобы не пропустить масл-кары:

In [ ]:
df = df[(df.power >= 50) & (df.power <= 2300)]

In [ ]:
histogram(df, 'car_mileage')
whiskers(df, 'car_mileage')

В этих данных все значения могут иметь место, ограничивать данные не будем

In [ ]:
histogram(df, 'registration_month')
whiskers(df, 'registration_month')

В году 12 месяцев, максимальное значение соответствует этой цифре. А вот "нулевого" месяца не бывает, исключим его из выборки:

In [ ]:
df = df[(df.registration_month >= 1)]

Снова выведем информацию о количественных признаках датасета:

In [ ]:
df.describe().round(2)

Изучим распределение ряда категориальных признаков:

In [ ]:
categorial(df, 'vehicle_type')

In [ ]:
categorial(df, 'gearbox')

In [ ]:
categorial(df, 'repaired')

Признаки `fuel_type` и `brand` построим на столбчатых диаграммах, чтобы данные визуально легко читались:

In [ ]:
df['fuel_type'].value_counts().plot(
    kind = 'bar',
    figsize = (5,5),
    title = 'Распределение признака "fuel_type"',
    ylabel = 'Количество'
);

In [ ]:
df['brand'].value_counts().plot(
    kind = 'bar',
    figsize = (10,10),
    title = 'Распределение признака "brand"',
    ylabel = 'Количество'
);

Выводы по разделу:

- исследовательский анализ данных выявил аномальные значения ряда признаков, диапазоны значений в некоторых из них были откорректированы с целью приведения к реально существующим;
- отбросили значения `price` ниже 1000 евро;
- диапазон `registration_year` сделали от 1900 г. до верхнего значения даты выгрузки данных (2017 г.);
- диапазон `power` сделали от 50 до 2300 л.с.;
- отбросили значение `registration_month`, равное "нулевому" месяцу;
- в тройке лидеров по признаку `vehicle_type`: "sedan" (29.8%), "wagon" (21.5%) и "small" (20.0%);
- подавляющее количество автомобилей - с механической КПП (77.2%) против 22.8% автомобилей на "автомате");
- автомобилей, никогда не бывавших в ремонте также большее количество - 93.1%;
- практически все автомобили - с бензиновыми двигателями;
- в пятёрке лидеров по признаку `brand` - исключительно "немцы": в порядке убывания - "Volkswagen", "BMW", "Mercedes-Benz", "Audi", "Opel".

## Корреляционный анализ признаков

Для построения тепловой карты коэффициента корреляции phik (используем именно его, т.к. корреляция Пирсона не подходит из-за наличия в датасете категориальных признаков, часть из которых имеет не нормальное распределение) исключим ряд признаков, очевидно не оказывающих никакого влияния на целевой признак `price`, а именно: `date_crawled`, `date_created`, `number_of_pictures`, `postal_code` и `last_seen`:

In [ ]:
corr_matrix = (
    df[df.columns.difference(['date_crawled', 'date_created', \
        'number_of_pictures', 'postal_code', 'last_seen'])] \
        .phik_matrix(interval_cols = ['price', 'power'])
)

In [ ]:
plot_correlation_matrix(corr_matrix.values, x_labels = corr_matrix.columns, \
                      y_labels=corr_matrix.index, color_map = 'coolwarm',
                      title = 'Тепловая карта коэффициентов корреляции phik', \
                        figsize = (7, 7))
plt.tight_layout()

Используя шкалу Чеддока, сопоставим тесноту связей между входными признаками и целевым `price` (0,1-0,3 - слабая; 0,3-0,5 - умеренная; 0,5-0,7 - заметная; 0,7-0,9 - высокая; 0,9-1 - весьма высокая):

`price`:
- теснота связи с `registration_year` = 0,63 (заметная),
- с `model` = 0,54 (заметная),
- с `car_mileage` = 0,32 (умеренная),
- с `brand` = 0,32 (умеренная),
- с `gearbox` = 0,30 (умеренная).

С остальными признаками связь слабая.

Дополнительно отметим, что между признаками `brand` и `model` наблюдается мультиколлинеарность, для обучения моделей в дальнейшем из этой пары будем использовать только `brand`.

Выводы по разделу:

- построили тепловую карту коэффициента корреляции phik, исключив из построения признаки `date_crawled`, `date_created`, `number_of_pictures`, `postal_code` и `last_seen`, как не оказывающие никакого значения на целевой признак `price`;
- используя шкалу Чеддока, определили тесноту связи между входными признаками и целевым: с `registration_year` = 0,63 (заметная), с `model` = 0,54 (заметная), с `car_mileage` = 0,32 (умеренная), с `brand` = 0,32 (умеренная), с `gearbox` = 0,30 (умеренная). С остальными признаками связь слабая;
- Обнаружена мультиколлинеарность между признаками `brand` и `model`, для обучения моделей оставим только `brand`.

## Обучение моделей

Прежде, чем приступить к обучению моделей, исключим из датасета признаки, не оказывающие влияния на целевой:

In [ ]:
df = df.drop(['date_crawled', 'date_created', 'number_of_pictures', \
            'postal_code', 'last_seen', 'registration_month'], axis = 1).copy()
df.info()

Самое время заполнить пропущенные значения в признаках, основываясь на изученных ранее данных:

In [ ]:
# Пропуски в признаке "vehicle_type" заполним значением "unknown":
df['vehicle_type'].fillna('unknown', inplace = True)

# Пропуски в признаке "gearbox" заполним значением "manual" (преобладающее):
df['gearbox'].fillna('manual', inplace = True)

# Пропуски в признаке "model" заполним значением "unknown":
df['model'].fillna('unknown', inplace = True)

# Пропуски в признаке "fuel_type" заполним значением "petrol" (преобладающее):
df['fuel_type'].fillna('petrol', inplace = True)

# Пропуски в признаке "brand" заполним значением "unknown":
df['brand'].fillna('unknown', inplace = True)

# Пропуски в признаке "repaired" заполним значением "no" (преобладающее):
df['repaired'].fillna('no', inplace = True)

Проверим, что пропуски отсутствуют:

In [ ]:
df.isna().sum()

Проверим также получившуюся таблицу на дубликаты:

In [ ]:
df.duplicated().sum()

Обнаружено 36550 дубликатов, удалим их:

In [ ]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

Модели готовы к обучению, будем использовать следующие: `LinearRegression`, `DecisionTree`, `LightGBM` и `CatBoost`. Подготовим данные, разбив на выборки. Напомним, что из обучения моделей исключили признак `model`, как мультиколлинеарный с `brand`:

In [ ]:
# Размер датасета перед сплитованием
df.shape[0]

In [ ]:
# Подготовим признаки и разделим данные на обучающую и тестовую выборки
features = df.drop(['price', 'model'], axis = 1)
target = df['price']

features_train, features_test, target_train, target_test = \
        train_test_split(features, target, test_size = TEST_SIZE, \
                         random_state = RANDOM_STATE)
print(f'Размер обучающей выборки: {features_train.shape[0]}')
print(f'Размер тестовой выборки: {target_test.shape[0]}')

Суммарно размеры совпадают, деление прошло успешно

In [ ]:
# Извлечём категориальные признаки из обучающего набора данных
cat_features = features_train.select_dtypes(include = 'object') \
                                                .columns.to_list()
# Извлечём числовые признаки из обучающего набора данных
num_features = features_train.select_dtypes(exclude = 'object') \
                                                .columns.to_list()

In [ ]:
# Создадим копию обучающего набора данных для нелинейных моделей
features_train_1 = features_train.copy()
# Создадим копию тестового набора данных для нелинейных моделей
features_test_1 = features_test.copy()

In [ ]:
# Трансформируем обучающую выборку для нелинейных моделей
col_transformer_1 = make_column_transformer(
    (OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features),
    (StandardScaler(), num_features),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [ ]:
features_train_transformed_1 = col_transformer_1.fit_transform(features_train_1)
features_test_transformed_1 = col_transformer_1.transform(features_test_1)

In [ ]:
# Создадим копии обучающего набора данных для линейной модели
features_train_2 = features_train.copy()
# Создадим копии тестового набора данных для линейной модели
features_test_2 = features_test.copy()

In [ ]:
# Трансформируем обучающую выборку для линейной модели
col_transformer_2 = make_column_transformer(
    (OneHotEncoder(drop='first', handle_unknown='ignore'), cat_features),
    (StandardScaler(), num_features),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [ ]:
features_train_transformed_2 = col_transformer_2.fit_transform(features_train_2)
features_test_transformed_2 = col_transformer_2.transform(features_test_2)

In [ ]:
# Функция подсчёта времени и RMSE для моделей
def model_param(model, parameters, features_train, target_train):
# Начинаем отсчет времени для поиска лучших параметров модели
    start_time = time.time()

# Инициализируем GridSearchCV для подбора гиперпараметров
    grid_cv = GridSearchCV(model, 
                        param_grid = parameters, 
                        cv=5, 
                        n_jobs=-1,
                        scoring = 'neg_root_mean_squared_error')
    
# Делаем фиттинг модели на обучающих данных с подбором параметров
    grid_cv.fit(features_train, target_train)

# Записываем время, затраченное на обучение модели
    time_1 = round((time.time() - start_time), 2)
      
# Ищем лучшую модель
    best_model = grid_cv.best_estimator_

    start_time = time.time()
    best_model.fit(features_train, target_train)
    time_best_model = round((time.time() - start_time), 2)
    
    
# Используем лучший результат из GridSearchCV как оценку RMSE
    rmse_cv = round(-grid_cv.best_score_, 2)

    # Сохраняем статистику модели
    model_stat = [time_1, time_best_model, rmse_cv]

    return model_stat, grid_cv

### Модель LinearRegression

In [ ]:
# Инициализируем модель
model_lr = LinearRegression(fit_intercept=True, \
                            copy_X=True, n_jobs=-1)

In [ ]:
# Гиперпараметры модели
param_grid_lr = {
    #'fit_intercept': [True, False],
    #'copy_X': [True, False]
}

In [ ]:
lr_stat, lr_grid_cv = model_param(model_lr, param_grid_lr, \
                                  features_train_transformed_2, target_train)

In [ ]:
lr_stat

In [ ]:
lr_grid_cv

### Модель DecisionTree

In [ ]:
# Инициализируем модель
model_tree = DecisionTreeRegressor(random_state = RANDOM_STATE)

In [ ]:
# Гиперпараметры модели
param_grid_tree = {
    'max_depth': range(2, 5),
    'min_samples_split': range(2, 5),
    'min_samples_leaf': range(2, 5)
}

In [ ]:
tree_stat, tree_grid_cv = model_param(model_tree, param_grid_tree, \
                                      features_train_transformed_1, target_train)

In [ ]:
tree_stat

In [ ]:
tree_grid_cv

### Модель LightGBM

In [ ]:
# Инициализируем модель
model_lgbm = LGBMRegressor(random_state = RANDOM_STATE)

In [ ]:
for col in cat_features:
    features_train_1[col] = features_train_1[col].astype('category')
    features_test_1[col] = features_test_1[col].astype('category')

In [ ]:
# Гиперпараметры модели
param_lgbm = {
    'num_leaves': range(2, 5),
    'max_depth': range(2, 5),
    'learning_rate': [0.01, 0.1, 0.2]
}

In [ ]:
lgbm_stat, lgbm_grid_cv = model_param(model_lgbm, param_lgbm, \
                                      features_train_transformed_1, target_train)

In [ ]:
lgbm_stat

In [ ]:
lgbm_grid_cv

### Модель CatBoost

In [ ]:
# Инициализируем модель
model_catboost = CatBoostRegressor(random_state = RANDOM_STATE)

In [ ]:
for col in cat_features:
    features_train_1[col] = features_train_1[col].astype(str)

In [ ]:
# Гиперпараметры модели
param_catboost = {
    'iterations': range(2, 5),
    'depth': range(2, 5),
    'learning_rate': [0.01, 0.1, 0.2]
}

In [ ]:
catboost_stat, catboost_grid_cv = model_param(model_catboost, param_catboost, \
                                            features_train_transformed_1, target_train)

In [ ]:
catboost_stat

In [ ]:
catboost_grid_cv

Соберём в единую таблицу данные от всех четырёх обученных моделей:

In [ ]:
model_param = pd.DataFrame(
    [lr_stat, tree_stat, lgbm_stat, catboost_stat],
    columns = ['Время обучения', 'Предсказанное время', 'RMSE'],
    index = ['LinearRegression', 'DecisionTree', 'LightGBM', \
             'CatBoost']
)
model_param

С заданным заказчиком условием "RMSE" <= 2500 справилась только модель `LightGBM`, её и будем рекомендовать к использованию, несмотря на самое долгое предсказанное время.

Проверим лучшую предсказанную модель на тестовой выборке:

In [ ]:
test_predict = lgbm_grid_cv.predict(features_test_transformed_1)

In [ ]:
test_rmse = round(np.sqrt(mean_squared_error(target_test, test_predict)), 2)
print(f'Значение RMSE на тестовой выборке: {test_rmse}')

Выводы по разделу:

- исключили из датасета признаки, не оказывающие влияния на целевой: `date_crawled`, `date_created`, `number_of_pictures`, `postal_code`, `last_seen` и `registration_month`;
- заполнили пропущенные значения в признаках: `vehicle_type`, `model`, `brand` - на "unknown", а в признаках `gearbox`, `fuel_type`, `repaired` - на преобладающие значения "manual", "petrol" и "no", соответственно;
- удалили дубликаты, образовавшиеся в обновлённом датасете.

Обучили 4 модели: `Linear Regression`, `Decision Tree`, `LightGBM` и `CatBoost`:
  - самый лучший показатель предсказанной метрики RMSE - у модели `LightGBM` (2057.15). Эта модель - единственная, удовлетворяющая условию заказчика (RMSE <= 2500). При этом LightGBM - самая медленная из обучаемых моделей;
  - проверили победившую модель на тестовой выборке, получив показатель RMSE = 2067.79, что также подходит под условия заказчика.

Итог: рекомендованная модель по результатам выполненного проекта - `LightGBM`.

## Отчёт по исследованию

В ходе выполнения проекта были проведены следующие этапы:

- загрузка данных. Изучили данные из предоставленного датасета, привели названия столбцов к единому виду (snake_case, строчные буквы); переименовали название столбца `Kilometer` в более подходящее по смыслу `car_mileage`; у столбцов `date_crawled`, `date_created` и `last_seen` поменяли тип данных на `datetime`, соответственно их значениям;

- предобработка данных. Проверили датасет на наличие пропущенных значений, приняли решение заполнить их позже; обнаружили 4 явных дубликата, удалили их; обнаружили одинаковые модели, записанные по-разному: `range_rover` и `rangerover`. Привели к единому виду; переименовали значение `gasoline` в `petrol` в столбце `fuel_type`, так как по сути это одно и то же. Разница - в стране употребления того или иного термина; неявные дубликаты обнаружены не были;

- исследовательский анализ данных. Исследовательский анализ данных выявил аномальные значения ряда признаков, диапазоны значений в некоторых из них были откорректированы с целью приведения к реально существующим, а именно:
  - отбросили значения `price` ниже 1000 евро;
  - диапазон `registration_year` сделали от 1900 г. до верхнего значения даты выгрузки данных (2017 г.);
  - диапазон `power` сделали от 50 до 2300 л.с.;
  - отбросили значение `registration_month`, равное "нулевому" месяцу; 

  в тройке лидеров по признаку `vehicle_type`: "sedan" (29.8%), "wagon" (21.5%) и "small" (20.0%); подавляющее количество автомобилей - с механической КПП (77.2%) против 22.8% автомобилей на "автомате"; автомобилей, никогда не бывавших в ремонте также большее количество - 93.1% практически все автомобили - с бензиновыми двигателями; в пятёрке лидеров по признаку `brand` - исключительно "немцы": в порядке убывания - "Volkswagen", "BMW", "Mercedes-Benz", "Audi", "Opel".

- корреляционный анализ признаков. Построили тепловую карту коэффициента корреляции phik, исключив из построения признаки `date_crawled`, `date_created`, `number_of_pictures`, `postal_code` и `last_seen`, как не оказывающие никакого значения на целевой признак `price`; используя шкалу Чеддока, определили тесноту связи между входными признаками и целевым: с `registration_year` = 0,63 (заметная), с `model` = 0,54 (заметная), с `car_mileage` = 0,32 (умеренная), с `brand` = 0,32 (умеренная), с `gearbox` = 0,30 (умеренная). С остальными признаками связь слабая; обнаружили мультиколлинеарность между признаками `brand` и `model`, для обучения моделей решили оставить только `brand`;

- обучение моделей. Исключили из датасета признаки, не оказывающие влияния на целевой `date_crawled`, `date_created`, `number_of_pictures`, `postal_code`, `last_seen` и `registration_month`; заполнили пропущенные значения в признаках: `vehicle_type`, `model`, `brand` - на "unknown", а в признаках `gearbox`, `fuel_type`, `repaired` - на преобладающие значения "manual", "petrol" и "no", соответственно; удалили дубликаты, образовавшиеся в обновлённом датасете.

- обучили 4 модели: `Linear Regression`, `Decision Tree`, `LightGBM` и `CatBoost`:
  - самый лучший показатель предсказанной метрики RMSE - у модели `LightGBM` (2057.15). Эта модель - единственная, удовлетворяющая условию заказчика (RMSE <= 2500). При этом LightGBM - самая медленная из обучаемых моделей;
  - проверили победившую модель на тестовой выборке, получив показатель RMSE = 2067.79, что также подходит под условия заказчика.

Итог: рекомендованная модель по результатам выполненного исследования - `LightGBM`.